In [74]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import kaleido

import pandas as pd
import json
import os

import scipy

from data_utils import DATA_PATH, YEARS
from collections import defaultdict

In [19]:
import os
os.environ["PATH"] = os.environ["PATH"] + ";venv\\lib\\site-packages\\kaleido\\executable\\"

In [80]:
state_abbreviations = '''Alabama	AL	Kentucky	KY	Ohio	OH
Alaska	AK	Louisiana	LA	Oklahoma	OK
Arizona	AZ	Maine	ME	Oregon	OR
Arkansas	AR	Maryland	MD	Pennsylvania	PA
American Samoa	AS	Massachusetts	MA	Puerto Rico	PR
California	CA	Michigan	MI	Rhode Island	RI
Colorado	CO	Minnesota	MN	South Carolina	SC
Connecticut	CT	Mississippi	MS	South Dakota	SD
Delaware	DE	Missouri	MO	Tennessee	TN
District of Columbia	DC	Montana	MT	Texas	TX
Florida	FL	Nebraska	NE	Trust Territories	TT
Georgia	GA	Nevada	NV	Utah	UT
Guam	GU	New Hampshire	NH	Vermont	VT
Hawaii	HI	New Jersey	NJ	Virginia	VA
Idaho	ID	New Mexico	NM	Virgin Islands	VI
Illinois	IL	New York	NY	Washington	WA
Indiana	IN	North Carolina	NC	West Virginia	WV
Iowa	IA	North Dakota	ND	Wisconsin	WI
Kansas	KS	Northern Mariana Islands	MP	Wyoming	WY'''
state_abbreviations = [token for line in state_abbreviations.split('\n') for token in line.split('\t')]
state_abbreviations = dict(zip([i for i in state_abbreviations[::2]], [i for i in state_abbreviations[1::2]]))
state_abbreviations.update({v: v for v in state_abbreviations.values()})
state_abbreviations.update({v.lower(): v for v in state_abbreviations.values()})
state_abbreviations.update({'U.S. Virgin Islands': 'VI', 'Il':"IL", 'United States Virgin Islands':"VI" ,'NMI':"MP", 'Virgin Islands of the United States':"VI",
                            "Tennessee;TN":"TN",
"Kentucky;Tennessee":"KY", # FIXME: this is a double location
"Ky":"KY",
                            "Commonwealth of the Northern Mariana Islands":"MP",
"Tx":"TX",
"Fl":"FL",'District Of Columbia':"DC"
                            })

In [10]:
iso3 = {"BD": "BGD", "BE": "BEL", "BF": "BFA", "BG": "BGR", "BA": "BIH", "BB": "BRB", "WF": "WLF", "BL": "BLM", "BM": "BMU", "BN": "BRN", "BO": "BOL", "BH": "BHR", "BI": "BDI", "BJ": "BEN", "BT": "BTN", "JM": "JAM", "BV": "BVT", "BW": "BWA", "WS": "WSM", "BQ": "BES", "BR": "BRA", "BS": "BHS", "JE": "JEY", "BY": "BLR", "BZ": "BLZ", "RU": "RUS", "RW": "RWA", "RS": "SRB", "TL": "TLS", "RE": "REU", "TM": "TKM", "TJ": "TJK", "RO": "ROU", "TK": "TKL", "GW": "GNB", "GU": "GUM", "GT": "GTM", "GS": "SGS", "GR": "GRC", "GQ": "GNQ", "GP": "GLP", "JP": "JPN", "GY": "GUY", "GG": "GGY", "GF": "GUF", "GE": "GEO", "GD": "GRD", "GB": "GBR", "GA": "GAB", "SV": "SLV", "GN": "GIN", "GM": "GMB", "GL": "GRL", "GI": "GIB", "GH": "GHA", "OM": "OMN", "TN": "TUN", "JO": "JOR", "HR": "HRV", "HT": "HTI", "HU": "HUN", "HK": "HKG", "HN": "HND", "HM": "HMD", "VE": "VEN", "PR": "PRI", "PS": "PSE", "PW": "PLW", "PT": "PRT", "SJ": "SJM", "PY": "PRY", "IQ": "IRQ", "PA": "PAN", "PF": "PYF", "PG": "PNG", "PE": "PER", "PK": "PAK", "PH": "PHL", "PN": "PCN", "PL": "POL", "PM": "SPM", "ZM": "ZMB", "EH": "ESH", "EE": "EST", "EG": "EGY", "ZA": "ZAF", "EC": "ECU", "IT": "ITA", "VN": "VNM", "SB": "SLB", "ET": "ETH", "SO": "SOM", "ZW": "ZWE", "SA": "SAU", "ES": "ESP", "ER": "ERI", "ME": "MNE", "MD": "MDA", "MG": "MDG", "MF": "MAF", "MA": "MAR", "MC": "MCO", "UZ": "UZB", "MM": "MMR", "ML": "MLI", "MO": "MAC", "MN": "MNG", "MH": "MHL", "MK": "MKD", "MU": "MUS", "MT": "MLT", "MW": "MWI", "MV": "MDV", "MQ": "MTQ", "MP": "MNP", "MS": "MSR", "MR": "MRT", "IM": "IMN", "UG": "UGA", "TZ": "TZA", "MY": "MYS", "MX": "MEX", "IL": "ISR", "FR": "FRA", "IO": "IOT", "SH": "SHN", "FI": "FIN", "FJ": "FJI", "FK": "FLK", "FM": "FSM", "FO": "FRO", "NI": "NIC", "NL": "NLD", "NO": "NOR", "NA": "NAM", "VU": "VUT", "NC": "NCL", "NE": "NER", "NF": "NFK", "NG": "NGA", "NZ": "NZL", "NP": "NPL", "NR": "NRU", "NU": "NIU", "CK": "COK", "XK": "XKX", "CI": "CIV", "CH": "CHE", "CO": "COL", "CN": "CHN", "CM": "CMR", "CL": "CHL", "CC": "CCK", "CA": "CAN", "CG": "COG", "CF": "CAF", "CD": "COD", "CZ": "CZE", "CY": "CYP", "CX": "CXR", "CR": "CRI", "CW": "CUW", "CV": "CPV", "CU": "CUB", "SZ": "SWZ", "SY": "SYR", "SX": "SXM", "KG": "KGZ", "KE": "KEN", "SS": "SSD", "SR": "SUR", "KI": "KIR", "KH": "KHM", "KN": "KNA", "KM": "COM", "ST": "STP", "SK": "SVK", "KR": "KOR", "SI": "SVN", "KP": "PRK", "KW": "KWT", "SN": "SEN", "SM": "SMR", "SL": "SLE", "SC": "SYC", "KZ": "KAZ", "KY": "CYM", "SG": "SGP", "SE": "SWE", "SD": "SDN", "DO": "DOM", "DM": "DMA", "DJ": "DJI", "DK": "DNK", "VG": "VGB", "DE": "DEU", "YE": "YEM", "DZ": "DZA", "US": "USA", "UY": "URY", "YT": "MYT", "UM": "UMI", "LB": "LBN", "LC": "LCA", "LA": "LAO", "TV": "TUV", "TW": "TWN", "TT": "TTO", "TR": "TUR", "LK": "LKA", "LI": "LIE", "LV": "LVA", "TO": "TON", "LT": "LTU", "LU": "LUX", "LR": "LBR", "LS": "LSO", "TH": "THA", "TF": "ATF", "TG": "TGO", "TD": "TCD", "TC": "TCA", "LY": "LBY", "VA": "VAT", "VC": "VCT", "AE": "ARE", "AD": "AND", "AG": "ATG", "AF": "AFG", "AI": "AIA", "VI": "VIR", "IS": "ISL", "IR": "IRN", "AM": "ARM", "AL": "ALB", "AO": "AGO", "AQ": "ATA", "AS": "ASM", "AR": "ARG", "AU": "AUS", "AT": "AUT", "AW": "ABW", "IN": "IND", "AX": "ALA", "AZ": "AZE", "IE": "IRL", "ID": "IDN", "UA": "UKR", "QA": "QAT", "MZ": "MOZ"}

In [21]:
def map(locations, year, user_type='followers'):
    us_locations = locations[locations.countrycode=='US'].copy()
    print(f'number of entries with a location in the US: {len(us_locations)}')
    us_locations.state = us_locations.state.dropna().apply(lambda x: state_abbreviations[x])
    us_location_counts = us_locations.groupby('state').size().to_frame('size')
    print(us_location_counts.sort_values('size', ascending=False))
    print("total entries with a state:", us_location_counts.sum().values[0])
    fig = px.choropleth(us_location_counts.reset_index(),
                        locations='state',
                        locationmode="USA-states",
                        scope="usa",
                        color='size',
                        color_continuous_scale="Blues",
                        title=f'{user_type} by U.S. state in {year}'
                        )
    fig.write_image(f"figures/{user_type.lower()}_{year}_usdmap.pdf")

    fig.show()

In [22]:
def map_world(year, locations, user_type):
    df = locations.groupby('countrycode').size().to_frame('size').reset_index().dropna()
    print(df.sort_values('size', ascending=False))
    fig = go.Figure(data=go.Choropleth(
        locations = df['countrycode'].map(iso3),
        z = df['size'],
        colorscale = 'Blues',
        autocolorscale=False,
        # reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        # colorbar_tickprefix = '$',
        colorbar_title = f'# {user_type.lower()}',
    ))

    fig.update_layout(
        title_text=f'{user_type} by country in {year}',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),

    )

    fig.write_image(f"figures/{user_type.lower()}_{year}_worldmap.pdf")
    fig.show()

In [23]:
out_dir = os.path.join(DATA_PATH, 'paper_data', 'meta')
with open(os.path.join(out_dir, 'twitter_follower_locations.json'), encoding='utf8') as f:
    twitter_follower_locations = json.load(f)
with open(os.path.join(out_dir, 'twitter_author_locations.json'), encoding='utf8') as f:
    twitter_author_locations = json.load(f)

In [24]:
for year in ['2016', '2020']:
    locations = pd.DataFrame(twitter_author_locations[year]).T
    map(locations, year, 'authors')

number of entries with a location in the US: 128
       size
state      
CA       13
TX        9
NY        8
FL        7
IL        7
UT        4
GA        4
TN        4
MA        4
AL        3
MO        3
OK        3
OH        3
AZ        3
WI        3
MN        3
DC        3
NC        2
NJ        2
NV        2
MI        2
KS        2
IN        2
OR        2
IA        1
WA        1
VA        1
PR        1
ID        1
SC        1
RI        1
PA        1
NE        1
total entries with a state: 107


number of entries with a location in the US: 174
       size
state      
TX       23
NY       17
FL       15
CA       14
AR        7
MN        7
MI        3
NV        3
NC        3
MD        3
GA        3
IN        3
IL        3
WI        3
NJ        2
TN        2
OH        2
CO        2
CT        2
NM        2
VA        2
LA        2
WA        2
UT        2
SC        2
MA        2
PA        2
MT        1
OK        1
ND        1
MO        1
KY        1
IA        1
DE        1
WV        1
total entries with a state: 141


In [25]:
for year in ['2016', '2020']:
    locations = pd.DataFrame(twitter_follower_locations[year]).T
    map(locations, year)

number of entries with a location in the US: 772470
         size
state        
CA     111224
TX      89711
FL      59358
NY      50040
OH      25877
IL      20989
GA      19573
AZ      16005
MA      14019
NC      13769
AL      13603
PA      13068
TN      11154
ID      10739
MN      10531
IN      10477
RI      10001
NV       9724
NJ       9641
WA       9539
OR       9427
MI       8906
DC       8767
CO       8556
NE       7706
MD       7356
AR       7319
VA       6694
MO       6335
WI       6118
OK       5981
UT       5828
KY       4813
SC       4796
LA       4286
IA       3949
KS       3891
MS       2721
NM       2530
AK       1590
CT       1582
HI       1579
WV       1513
NH       1370
MT       1275
PR       1265
ND       1021
ME        920
SD        918
WY        848
DE        819
VT        786
VI         77
GU         40
AS         24
MP         20
total entries with a state: 660668


number of entries with a location in the US: 775878
        size
state       
CA     94972
NY     60779
FL     57366
TX     55105
IL     27162
OH     25300
MN     19302
GA     18435
PA     18375
WI     16466
NC     15648
IN     14688
TN     13011
AZ     12714
NJ     12501
MA     11746
NV     11744
WA     11591
AR     11429
CO     10384
MI     10150
MD     10132
AL     10002
VA      9805
DC      9404
OR      8810
MO      7897
OK      7886
KY      6954
SC      6883
LA      6316
KS      5713
UT      5064
IA      4563
MS      3848
NM      3220
NE      2763
AK      2622
WV      2217
HI      2183
ID      2068
NH      1966
PR      1893
CT      1657
MT      1527
ND      1348
ME      1195
SD      1111
WY      1093
VT      1068
RI      1008
DE       981
GU       105
VI        78
MP        32
AS        18
total entries with a state: 662298


In [26]:
for year in ['2016', '2020']:
    locations = pd.DataFrame(twitter_follower_locations[year]).T
    user_type = 'followers'
    map_world(year, locations, user_type)

    countrycode    size
202          US  775878
153          PH  773561
141          NG  329895
66           GB  325629
89           IN  108622
..          ...     ...
55           EH      23
156          PN      13
132          MS      11
147          NU       7
190          TK       6

[217 rows x 2 columns]


    countrycode    size
201          US  772470
24           BR  166884
66           GB  131093
30           CA  128859
89           IN  103070
..          ...     ...
159          PW      10
132          MS       9
196          TV       4
156          PN       3
147          NU       2

[216 rows x 2 columns]


    countrycode    size
202          US  775878
153          PH  773561
141          NG  329895
66           GB  325629
89           IN  108622
..          ...     ...
55           EH      23
156          PN      13
132          MS      11
147          NU       7
190          TK       6

[217 rows x 2 columns]


In [27]:
for year in ['2016', '2020']:
    locations = pd.DataFrame(twitter_author_locations[year]).T
    user_type = 'authors'
    map_world(year, locations, user_type)

   countrycode  size
39          US   128
14          GB    32
7           CA    15
13          FR     8
36          TR     6
27          NL     4
19          IT     4
17          IN     4
10          DE     4
43          ZA     4
2           AU     4
16          IL     3
25          MX     3
28          NO     2
32          PL     2
12          ES     2
41          VE     2
5           BR     2
4           BE     2
24          LV     2
34          SE     2
31          PH     1
40          UZ     1
33          PY     1
35          SO     1
38          UA     1
37          TT     1
30          PE     1
42          VN     1
0           AE     1
29          OM     1
26          MY     1
23          LU     1
1           AR     1
21          JP     1
20          JO     1
18          IQ     1
15          GL     1
11          DK     1
9           CL     1
8           CG     1
6           BY     1
3           AZ     1
22          KE     1


   countrycode  size
53          US   174
17          GB    41
6           CA    21
48          TR    20
24          IT    18
16          FR    16
22          IN    16
11          DE    10
2           AU     9
15          ES     8
36          NG     8
55          ZA     8
1           AR     7
39          PA     6
37          NL     4
42          PL     4
5           BR     4
51          UA     4
41          PK     3
34          MX     3
40          PH     3
28          LB     3
21          IE     2
14          EC     2
3           BE     2
7           CH     2
44          SA     2
9           CO     2
10          CR     2
23          IR     2
12          DK     2
20          ID     2
29          LU     2
27          KE     2
31          MC     1
45          SE     1
54          XK     1
52          UG     1
4           BG     1
50          TZ     1
49          TT     1
25          JE     1
47          TC     1
46          SL     1
43          RW     1
26          JP     1
32          M

In [28]:
election_outcomes = pd.read_csv(os.path.join(out_dir, '1976-2020-president.csv'))
election_outcomes = election_outcomes[election_outcomes.year.isin([2016, 2020])]
election_outcomes = election_outcomes[election_outcomes.candidate=='TRUMP, DONALD J.']
election_outcomes['trump_pct'] = 100*election_outcomes.candidatevotes/election_outcomes.totalvotes
election_outcomes[['state_po', 'year' , 'trump_pct']]

,state_po,year,trump_pct
3395,AL,2016,62.083092
3400,AK,2016,51.281512
3407,AZ,2016,48.671616
3417,AR,2016,60.574102
3426,CA,2016,31.617107
...,...,...,...
4253,VA,2020,43.995504
4257,WA,2020,38.766978
4264,WV,2020,68.631552
4268,WI,2020,48.822437


In [91]:
for year in sorted(election_outcomes.year.unique()):
    fig = px.choropleth(election_outcomes[election_outcomes.year==year].reset_index(),
                        locations='state_po',
                        locationmode="USA-states",
                        scope="usa",
                        color='trump_pct',
                        color_continuous_scale=px.colors.sequential.Blues,
                        # color_continuous_scale=px.colors.sequential.RdBu_r,
                        color_continuous_midpoint=50.0,
                        range_color=[0, 100],
                        title=f'% votes for Trump in {year}'
                        )
    fig.write_image(f"figures/{year}_election_outcomes.pdf")

    fig.show()

In [32]:
with open(os.path.join(out_dir, 'twitter_author_ids_and_trump_percent.json'), encoding='utf8') as f:
    twitter_author_ids_and_trump_percent = json.load(f)

In [92]:
for year in sorted(twitter_author_locations):
    author_locations = pd.DataFrame(twitter_author_locations[year]).T

    df = pd.DataFrame(twitter_author_ids_and_trump_percent)[year].dropna().explode().to_frame().reset_index()
    df.columns = ['author_id', 'trump_pct']
    # df = df.groupby('author_id').mean()

    author_locations = pd.merge(author_locations, df, left_index=True, right_on='author_id', how='right')

    us_locations = author_locations[author_locations.countrycode=='US'].copy()
    us_locations.dropna(subset=['state'], inplace=True)
    us_locations.state = us_locations.state.apply(lambda x: state_abbreviations[x])
    us_locations = us_locations.groupby('state').trump_pct.mean().reset_index()
    print(us_locations[['state', 'trump_pct']].sort_values('trump_pct'))
    election_outcomes_ = election_outcomes[election_outcomes.year==int(year)].reset_index()[['state', 'trump_pct']].rename(columns={'trump_pct':'trump_pct_elections'})
    election_outcomes_.state = election_outcomes_.state.apply(lambda x:state_abbreviations[x.title()])
    tocorr = pd.merge(us_locations[['state', 'trump_pct']].sort_values('trump_pct'), election_outcomes_, on='state')
    tocorr.set_index('state', inplace=True)
    r, p = scipy.stats.pearsonr(tocorr.trump_pct, tocorr.trump_pct_elections)
    fig = px.choropleth(us_locations,
                        locations='state',
                        locationmode="USA-states",
                        scope="usa",
                        color='trump_pct',
                        color_continuous_scale=px.colors.sequential.Blues,
                        # color_continuous_scale=px.colors.sequential.RdBu_r,
                        color_continuous_midpoint=50.0,
                        range_color=[0, 100],
                        title=f'avg % preference for Trump in {year} in twitter polls (corr={r:.2f} p={p:.3f})'
                        )
    fig.write_image(f"figures/{year}_twitter_poll_outcomes.pdf")

    fig.show()


   state  trump_pct
16    NE  31.313131
22    OR  32.737223
26    SC  33.663366
1     AZ  39.171843
14    MO  39.450252
23    PA  40.000000
10    KS  42.741935
25    RI  42.857143
5     GA  44.305949
7     ID  48.936170
28    TX  49.850150
30    VA  50.000000
15    NC  51.381815
32    WI  52.387539
20    OH  52.810847
18    NV  53.095975
11    MA  53.678599
4     FL  53.924208
3     DC  54.092105
2     CA  54.230567
19    NY  55.993804
24    PR  56.344086
8     IL  58.326306
21    OK  63.840580
9     IN  64.774298
13    MN  66.717455
29    UT  71.186897
17    NJ  72.430759
6     IA  74.285714
31    WA  74.432761
27    TN  74.713065
0     AL  77.275986
12    MI  81.666667


   state  trump_pct
32    WA  25.480769
31    VA  29.334023
7     IA  30.000000
22    NV  33.007072
4     DE  33.333333
20    NJ  33.536585
8     IL  39.841321
9     IN  43.045412
5     FL  44.534326
14    MI  45.182084
16    MO  46.296296
12    MA  46.458269
26    PA  46.560197
0     AR  47.276335
23    NY  47.290909
19    ND  50.000000
10    KY  50.000000
3     CT  52.363636
18    NC  52.470862
1     CA  58.237250
29    TX  58.978588
30    UT  60.576923
21    NM  62.315254
13    MD  62.893082
2     CO  63.552632
28    TN  64.092664
6     GA  64.152047
27    SC  65.873016
33    WI  70.604472
24    OH  80.071225
11    LA  81.547619
25    OK  85.207101
15    MN  88.785376
17    MT  89.285714
34    WV  94.285714


In [93]:
for year in sorted(twitter_author_locations):
    author_locations = pd.DataFrame(twitter_author_locations[year]).T

    df = pd.DataFrame(twitter_author_ids_and_trump_percent)[year].dropna().explode().to_frame().reset_index()
    df.columns = ['author_id', 'trump_pct']
    df = df.groupby('author_id').mean().reset_index()

    author_locations = pd.merge(author_locations, df, left_index=True, right_on='author_id', how='right')

    us_locations = author_locations[author_locations.countrycode=='US'].copy()
    us_locations.dropna(subset=['state'], inplace=True)
    us_locations.state = us_locations.state.apply(lambda x: state_abbreviations[x])
    # us_locations = us_locations.groupby('state').trump_pct.mean().reset_index()
    print(us_locations[['state', 'trump_pct']].sort_values('trump_pct'))
    election_outcomes_ = election_outcomes[election_outcomes.year==int(year)].reset_index()[['state', 'trump_pct']].rename(columns={'trump_pct':'trump_pct_elections'})
    election_outcomes_.state = election_outcomes_.state.apply(lambda x:state_abbreviations[x.title()])
    tocorr = pd.merge(us_locations[['state', 'trump_pct']].sort_values('trump_pct'), election_outcomes_, on='state')
    tocorr.set_index('state', inplace=True)
    r, p = scipy.stats.pearsonr(tocorr.trump_pct, tocorr.trump_pct_elections)
    fig = px.choropleth(us_locations,
                        locations='state',
                        locationmode="USA-states",
                        scope="usa",
                        color='trump_pct',
                        color_continuous_scale=px.colors.sequential.Blues,
                        # color_continuous_scale=px.colors.sequential.RdBu_r,
                        color_continuous_midpoint=50.0,
                        range_color=[0, 100],
                        title=f'avg % preference for Trump in {year} for twitter poll authors (corr={r:.2f} p={p:.3f})'
                        )
    fig.write_image(f"figures/{year}_twitter_author_preferences.pdf")

    fig.show()


    state  trump_pct
210    MO   0.000000
55     TX   0.000000
264    NY  10.714286
122    MA  14.285714
202    GA  16.091954
..    ...        ...
73     TN  85.470085
32     GA  87.500000
140    TX  88.811189
119    MA  89.232753
296    NY  97.619048

[107 rows x 2 columns]


    state  trump_pct
333    NY   0.000000
337    FL   0.000000
150    AR   0.000000
462    NC   0.000000
176    MN   0.000000
..    ...        ...
523    MN  91.532546
598    SC  91.666667
627    LA  91.666667
433    TX  94.047619
428    WV  94.285714

[141 rows x 2 columns]


In [94]:
with open(os.path.join(out_dir, 'twitter_follower_lists.json'), encoding='utf8') as f:
    twitter_follower_lists = json.load(f)

In [95]:
for year in ['2016', '2020']:


    follower_to_author_id = defaultdict(set)
    for author, followers in twitter_follower_lists[year].items():
        for follower in followers:
            follower_to_author_id[follower].add(author)
    follower_to_trump_percent = defaultdict(list)
    for follower, authors in follower_to_author_id.items():
        for author in authors:
            follower_to_trump_percent[follower].extend(twitter_author_ids_and_trump_percent[year][author])
    follower_to_trump_percent = {k: np.mean(v) for k, v in follower_to_trump_percent.items()}
    follower_locations = pd.DataFrame(twitter_follower_locations[year]).T

    df = pd.DataFrame(follower_to_trump_percent.items())
    df.columns = ['follower_id', 'trump_pct']
    # df = df.groupby('follower_id').mean().reset_index()

    follower_locations = pd.merge(follower_locations, df, left_index=True, right_on='follower_id', how='right')

    us_locations = follower_locations[follower_locations.countrycode=='US'].copy()
    us_locations.dropna(subset=['state'], inplace=True)
    us_locations.state = us_locations.state.apply(lambda x: state_abbreviations[x])
    us_locations = us_locations.groupby('state').trump_pct.mean().reset_index()
    print(us_locations[['state', 'trump_pct']].sort_values('trump_pct'))
    election_outcomes_ = election_outcomes[election_outcomes.year==int(year)].reset_index()[['state', 'trump_pct']].rename(columns={'trump_pct':'trump_pct_elections'})
    election_outcomes_.state = election_outcomes_.state.apply(lambda x:state_abbreviations[x.title()])
    tocorr = pd.merge(us_locations[['state', 'trump_pct']].sort_values('trump_pct'), election_outcomes_, on='state')
    tocorr.set_index('state', inplace=True)
    r, p = scipy.stats.pearsonr(tocorr.trump_pct, tocorr.trump_pct_elections)

    fig = px.choropleth(us_locations,
                        locations='state',
                        locationmode="USA-states",
                        scope="usa",
                        color='trump_pct',
                        color_continuous_scale=px.colors.sequential.Blues,
                        # color_continuous_scale=px.colors.sequential.RdBu_r,
                        color_continuous_midpoint=50.0,
                        range_color=[30, 70],
                        title=f'avg % preference for Trump in {year} for twitter poll followers (corr={r:.2f} p={p:.3f})'
                        )
    fig.write_image(f"figures/{year}_twitter_follower_preferences.pdf")

    fig.show()

   state  trump_pct
32    NE  39.291819
43    RI  44.004828
40    OR  50.134347
5     CA  50.215180
15    ID  50.906665
47    TX  50.984491
21    MA  52.097452
4     AZ  53.311334
42    PR  53.843070
12    GU  54.090875
16    IL  55.301448
27    MP  55.479906
14    IA  55.578206
8     DC  56.185440
7     CT  56.204643
3     AS  56.514491
38    OH  57.039668
37    NY  57.072304
31    ND  57.285970
25    MN  57.356432
23    ME  57.393036
55    WY  57.415672
11    GA  57.467135
45    SD  57.477899
22    MD  57.676588
51    VT  57.739371
0     AK  57.989329
18    KS  58.158913
29    MT  58.267054
36    NV  58.411435
52    WA  58.479066
24    MI  58.487424
6     CO  58.590490
35    NM  58.603446
53    WI  58.612235
13    HI  58.643734
26    MO  58.744851
39    OK  59.005694
9     DE  59.456985
34    NJ  59.593379
49    VA  59.660033
17    IN  59.952498
33    NH  59.999666
19    KY  60.074298
41    PA  60.134592
2     AR  60.525578
54    WV  60.632693
30    NC  60.697775
20    LA  60.873242


   state  trump_pct
27    MP  33.042380
12    GU  34.108171
42    PR  41.445302
5     CA  42.376743
13    HI  43.058929
23    ME  43.142324
0     AK  43.799164
25    MN  43.896955
3     AS  44.007768
16    IL  44.118726
22    MD  44.185348
31    ND  45.202293
49    VA  45.300956
52    WA  45.372470
18    KS  45.528258
51    VT  45.726218
35    NM  46.046508
9     DE  46.049831
2     AR  46.270973
24    MI  46.417058
37    NY  46.432153
14    IA  46.631714
8     DC  46.991021
21    MA  47.174395
48    UT  47.833536
53    WI  47.863527
19    KY  48.020882
20    LA  48.067526
7     CT  48.277192
45    SD  48.369437
30    NC  48.416066
26    MO  48.425462
32    NE  48.449259
10    FL  48.483880
34    NJ  48.492095
1     AL  48.521822
55    WY  48.631494
28    MS  48.659716
41    PA  48.663136
29    MT  48.675985
38    OH  48.702285
36    NV  48.732597
17    IN  48.738753
33    NH  48.748430
39    OK  48.944461
40    OR  49.001423
6     CO  49.026653
43    RI  49.069128
11    GA  49.349635
